# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [37]:
city_data=pd.read_csv("../output_data/city_weather_data.csv")
city_data.head()


,City,Latitude,Longitude,Maximum Temp,Humidity,Clouds,Wind Speed,Country,Date
0,sioux lookout,50.10,-91.92,69.80,49,40,10.29,CA,1597077060
1,asfi,32.30,-9.24,73.63,74,48,20.54,MA,1597077158
2,tateyama,34.98,139.87,84.00,89,5,4.00,JP,1597077158
3,vostok,46.49,135.88,60.40,100,83,3.69,RU,1597077158
4,rikitea,-23.12,-134.97,69.84,74,47,19.33,PF,1597077051


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [44]:
gmaps.configure(api_key=g_key)

In [45]:
locations=city_data[['Latitude','Longitude']]
humidity=city_data['Humidity']
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable
heatlayer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=300,point_radius=5)
# Add the layer to the map
fig.add_layer(heatlayer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#filter df by ideal conditions 
ideal_condition_city=city_data.loc[(city_data['Maximum Temp']<80)&\
                            (city_data["Maximum Temp"]>70)&\
                           (city_data['Wind Speed']<10)&\
                           (city_data['Clouds']==0)].dropna()
ideal_condition_city

,City,Latitude,Longitude,Maximum Temp,Humidity,Clouds,Wind Speed,Country,Date
28,krapivinskiy,55.00,86.81,71.60,56,0,6.71,RU,1597077160
46,lingyuan,41.24,119.40,70.09,39,0,4.43,CN,1597077161
78,cidreira,-30.18,-50.21,77.36,47,0,7.52,BR,1597077067
129,weifang,36.71,119.10,77.22,87,0,6.29,CN,1597077167
153,castro,-24.79,-50.01,73.51,44,0,8.63,BR,1597077047
167,shirochanka,46.65,38.40,78.98,41,0,8.39,RU,1597077169
169,sao sebastiao do paraiso,-20.92,-46.99,78.30,32,0,5.61,BR,1597077170
191,avarua,-21.21,-159.78,71.60,73,0,6.93,CK,1597077047
214,laguna,38.42,-121.42,75.99,52,0,1.01,US,1597077172
338,kalabo,-14.97,22.68,72.86,20,0,5.95,ZM,1597077180


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
#add column for list of hotels 
hotel_df=ideal_condition_city[['City','Country','Latitude','Longitude']].copy()
hotel_df['Hotel Name']=''
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
28,krapivinskiy,RU,55.00,86.81,
46,lingyuan,CN,41.24,119.40,
78,cidreira,BR,-30.18,-50.21,
129,weifang,CN,36.71,119.10,
153,castro,BR,-24.79,-50.01,
167,shirochanka,RU,46.65,38.40,
169,sao sebastiao do paraiso,BR,-20.92,-46.99,
191,avarua,CK,-21.21,-159.78,
214,laguna,US,38.42,-121.42,
338,kalabo,ZM,-14.97,22.68,


In [48]:
#https://maps.googleapis.com/maps/api/place/nearbysearch/json

params={'radius':5000,'types':'lodging','key':g_key}

for index,row in hotel_df.iterrows():
    lat=row['Latitude']
    lng=row['Longitude']
    params["location"]={lat},{lng}
    url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    address=requests.get(url,params=params).json()
                         
    try:
        hotel_df.loc[index,'Hotel Name']=address['results'][0]['name']
    except (KeyError,IndexError):
        print("Nothing Found....Skipping")

hotel_df
                 

Nothing Found....Skipping
Nothing Found....Skipping
Nothing Found....Skipping
Nothing Found....Skipping
Nothing Found....Skipping
Nothing Found....Skipping
Nothing Found....Skipping
Nothing Found....Skipping
Nothing Found....Skipping
Nothing Found....Skipping
Nothing Found....Skipping
Nothing Found....Skipping
Nothing Found....Skipping
Nothing Found....Skipping
Nothing Found....Skipping


,City,Country,Latitude,Longitude,Hotel Name
28,krapivinskiy,RU,55.00,86.81,
46,lingyuan,CN,41.24,119.40,Guomao Hotel
78,cidreira,BR,-30.18,-50.21,Novomar Pousada e Turismo Ltda
129,weifang,CN,36.71,119.10,潍坊国际金融大酒店
153,castro,BR,-24.79,-50.01,CHACARA BAILLY
167,shirochanka,RU,46.65,38.40,Guest House Larisa
169,sao sebastiao do paraiso,BR,-20.92,-46.99,BM Hotel
191,avarua,CK,-21.21,-159.78,Paradise Inn
214,laguna,US,38.42,-121.42,Holiday Inn Express & Suites Elk Grove Central...
338,kalabo,ZM,-14.97,22.68,Sikakubete Guest House Kalabo Town Council


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [18]:
# Add marker layer ontop of heat map
# Add marker layer ontop of heat map
# Assign the marker layer to a variable

    
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)


# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…